In [1]:
import pandas as pd
from sodapy import Socrata
import pyspark
import json

conf = pyspark.SparkConf()
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)

client = Socrata("data.cityofnewyork.us", None)

23/05/02 19:26:50 WARN Utils: Your hostname, MacBook-Air-2.local resolves to a loopback address: 127.0.0.1; using 192.168.0.216 instead (on interface en0)
23/05/02 19:26:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/02 19:26:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/02 19:26:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/05/02 19:26:51 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
data_2014 = 'jt7v-77mi'
data_2015 = 'c284-tqph'
data_2016 = 'kiv2-tbus'
data_2016 = 'uu62-ifvu'
data_2017 = '2bnn-yakx'
data_2018 = 'a5td-mswe'
data_2019 = 'faiq-9dfq'
data_2020 = 'p7t3-5i9s'
data_2021 = 'kvfd-bves'
data_2022 = '7mxj-7a6y'
data_2023 = 'pvqr-7yc4'

main_data = "nc67-uf89"

In [3]:
# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
res_list = []
offset = 0
for i in range(2):    
    results = client.get(main_data, limit=50000, offset=offset)
    res_list.append(results)
    offset = offset + 50000

In [4]:
df = sc.parallelize(res_list).map(lambda x: json.dumps(x))
df = spark.read.json(df)

23/05/02 19:27:04 WARN TaskSetManager: Stage 0 contains a task of very large size (14993 KiB). The maximum recommended task size is 1000 KiB.


In [5]:
df.printSchema()

root
 |-- amount_due: string (nullable = true)
 |-- county: string (nullable = true)
 |-- fine_amount: string (nullable = true)
 |-- interest_amount: string (nullable = true)
 |-- issue_date: string (nullable = true)
 |-- issuing_agency: string (nullable = true)
 |-- judgment_entry_date: string (nullable = true)
 |-- license_type: string (nullable = true)
 |-- payment_amount: string (nullable = true)
 |-- penalty_amount: string (nullable = true)
 |-- plate: string (nullable = true)
 |-- precinct: string (nullable = true)
 |-- reduction_amount: string (nullable = true)
 |-- state: string (nullable = true)
 |-- summons_image: struct (nullable = true)
 |    |-- description: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- summons_number: string (nullable = true)
 |-- violation: string (nullable = true)
 |-- violation_status: string (nullable = true)
 |-- violation_time: string (nullable = true)



In [6]:
df.limit(3).toPandas()

23/05/02 19:27:12 WARN TaskSetManager: Stage 2 contains a task of very large size (14993 KiB). The maximum recommended task size is 1000 KiB.


,amount_due,county,fine_amount,interest_amount,issue_date,issuing_agency,judgment_entry_date,license_type,payment_amount,penalty_amount,plate,precinct,reduction_amount,state,summons_image,summons_number,violation,violation_status,violation_time
0,0,K,35,0,08/03/2018,TRAFFIC,None,PAS,35,0,FFC4982,060,0,NY,"(View Summons, http://nycserv.nyc.gov/NYCServW...",8661033100,FAIL TO DSPLY MUNI METER RECPT,None,08:16P
1,0,BX,35,0,04/27/2018,TRAFFIC,None,PAS,65,30,BTM5131,049,0,NY,"(View Summons, http://nycserv.nyc.gov/NYCServW...",8541152224,FAIL TO DSPLY MUNI METER RECPT,HEARING HELD-GUILTY,01:15P
2,0,BX,35,0,04/30/2018,TRAFFIC,None,PAS,95,60,HNY4992,047,0,NY,"(View Summons, http://nycserv.nyc.gov/NYCServW...",8541152327,FAIL TO DSPLY MUNI METER RECPT,None,01:14P


In [7]:
df.count()

23/05/02 19:27:15 WARN TaskSetManager: Stage 3 contains a task of very large size (14993 KiB). The maximum recommended task size is 1000 KiB.


100000